In [1]:
"""
# Description: Consolidates all standardized and enriched datasets into a
#              point, line, and polygon feature class.
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
import os
import datetime
import arcpy
from scripts._2b_assign_domains import AssignDomains
from scripts.utils import init_gdb, og_file_input 
from scripts._4_metadata_append import pt_metadata, ln_metadata, poly_metadata

original_gdb, workspace, scratch_workspace = init_gdb()
date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216

# Reference
WFR_TF_Template = os.path.join(workspace,'b_Reference','WFR_TF_Template')


In [2]:
## POINTS
# INPUTS
nfpors_pt = og_file_input(prefix = 'nfpors_fuels_treatments_pts_enriched_', filetype = 'Point', gdb = os.path.join(workspace, 'd_Enriched'))
nfpors_fuels_treatments_pts_enriched = os.path.join(workspace,'d_Enriched', nfpors_pt)

caltrans_pt = og_file_input(prefix = 'CalTrans_act_pts_enriched_', filetype = 'Point', gdb = os.path.join(workspace, 'd_Enriched'))
CalTrans_act_pts_enriched = os.path.join(workspace,'d_Enriched',caltrans_pt)

pfirs_pt = og_file_input(prefix = 'PFIRS_enriched_', filetype = 'Point', gdb = os.path.join(workspace, 'd_Enriched'))
PFIRS_enriched = os.path.join(workspace,'d_Enriched', pfirs_pt)

cnra_pt = og_file_input(prefix = 'CNRA_Enriched_pt_', filetype = 'Point', gdb = os.path.join(workspace, 'd_Enriched'))
CNRA_Enriched_pt = os.path.join(workspace,'d_Enriched',cnra_pt)

# Model Environment settings
with arcpy.EnvManager(
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="""450000, -374900, 540100, -604500""",
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        scratchWorkspace=scratch_workspace, 
        transferDomains=False, 
        transferGDBAttributeProperties=True, 
        workspace=workspace,
        overwriteOutput = True,
):
    
    # Process: Create Point Feature Class (Create Feature Class) (management)
    points = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'e_Appended'), 
        out_name=f"Treat_n_harvests_points_{date_id}", 
        geometry_type="POINT", 
        template=[WFR_TF_Template], 
        has_m="DISABLED", 
        has_z="DISABLED", 
        spatial_reference="NAD 1983 California (Teale) Albers (Meters)", 
        config_keyword="", 
        spatial_grid_1=0, 
        spatial_grid_2=0, 
        spatial_grid_3=0, 
        out_alias=""
        )

    # Process: Append Points (Append) (management)
    point_append = arcpy.management.Append(
        inputs=[nfpors_fuels_treatments_pts_enriched, 
                CalTrans_act_pts_enriched, 
                CNRA_Enriched_pt, 
                PFIRS_enriched
                ], 
        target=points, 
        schema_type="TEST", 
        field_mapping="", 
        subtype="", expression=""
        )

    # Process: 2b Assign Domains (2) (2b Assign Domains) (PC414CWIMillionAcres)
    point_append_ad = AssignDomains(in_table=point_append)

    # Assign Metadata to the featureclass
    point_append_md = pt_metadata(point_append_ad)    


Point Metadata added successfully.


In [3]:
##LINES
# Inputs
caltrans_ln = og_file_input(prefix = 'CalTrans_act_ln_enriched_', filetype = 'Line', gdb = os.path.join(workspace, 'd_Enriched'))
CalTrans_act_ln_enriched = os.path.join(workspace,'d_Enriched', caltrans_ln)

cnra_ln = og_file_input(prefix = 'CNRA_Enriched_ln_', filetype = 'Line', gdb = os.path.join(workspace, 'd_Enriched'))
CNRA_Enriched_ln = os.path.join(workspace,'d_Enriched',cnra_ln)

# Model Environment settings
with arcpy.EnvManager(
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="""450000, -374900, 540100, -604500""",
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        scratchWorkspace=scratch_workspace, 
        transferDomains=True, 
        transferGDBAttributeProperties=True, 
        workspace=workspace,
        overwriteOutput = True,
):
    
    # Process: Create Line Feature Class (Create Feature Class) (management)
    lines = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'e_Appended'), 
        out_name=f"Treat_n_harvests_lines_{date_id}", 
        geometry_type="POLYLINE", 
        template=[WFR_TF_Template], 
        has_m="DISABLED", 
        has_z="DISABLED", 
        spatial_reference="NAD 1983 California (Teale) Albers (Meters)", 
        config_keyword="", 
        spatial_grid_1=0, 
        spatial_grid_2=0, 
        spatial_grid_3=0, 
        out_alias=""
    )

    # Process: Append Lines (Append) (management)
    lines_append = arcpy.management.Append(
        inputs=[CalTrans_act_ln_enriched, 
                CNRA_Enriched_ln
                ], 
        target=lines, 
        schema_type="TEST", 
        field_mapping="", 
        subtype="", 
        expression=""
        )

    # Process: 2b Assign Domains (2b Assign Domains) (PC414CWIMillionAcres)
    lines_append_ad = AssignDomains(in_table=lines_append)

    # Assign Metadata to the featureclass
    lines_append_md = ln_metadata(lines_append_ad) 


Line Metadata added successfully.


In [4]:
## POLYGON
# Inputs
nfpors_poly = og_file_input(prefix = 'nfpors_fuels_treatments_enriched_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
nfpors_fuels_treatments_enriched = os.path.join(workspace,'d_Enriched', nfpors_poly)

tahoe_poly = og_file_input(prefix = 'Tahoe_Forest_Fuels_Tx_enriched_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
Tahoe_Forest_Fuels_Tx_enriched = os.path.join(workspace,'d_Enriched',tahoe_poly)

usfs_poly = og_file_input(prefix = 'usfs_edw_facts_common_attributes_enriched_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
usfs_edw_facts_common_attributes_enriched = os.path.join(workspace,'d_Enriched',usfs_poly)

nps_poly = og_file_input(prefix = 'nps_flat_fuels_enriched_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
nps_flat_fuels_enriched = os.path.join(workspace,'d_Enriched',nps_poly)

cnra_poly = og_file_input(prefix = 'CNRA_Enriched_poly_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
CNRA_Enriched_poly = os.path.join(workspace,'d_Enriched',cnra_poly)

blm_poly = og_file_input(prefix = 'BLM_enriched_', filetype = 'Polygon', gdb = os.path.join(workspace, 'd_Enriched'))
BLM_enriched = os.path.join(workspace,'d_Enriched',blm_poly)

# Model Environment settings
with arcpy.EnvManager(
        outputCoordinateSystem= arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        cartographicCoordinateSystem=arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)"), #WKID 3310
        extent="""450000, -374900, 540100, -604500""",
        preserveGlobalIds=True, 
        qualifiedFieldNames=False, 
        scratchWorkspace=scratch_workspace, 
        transferDomains=True, 
        transferGDBAttributeProperties=True, 
        workspace=workspace,
        overwriteOutput = True,
):

    # Process: Create Polygon Feature Class (Create Feature Class) (management)
    polygons = arcpy.management.CreateFeatureclass(
        out_path=os.path.join(workspace,'e_Appended'), 
        out_name=f"Treat_n_harvests_polygons_{date_id}", 
        geometry_type="POLYGON", 
        template=[WFR_TF_Template], 
        has_m="DISABLED", 
        has_z="DISABLED", 
        spatial_reference="NAD 1983 California (Teale) Albers (Meters)", 
        config_keyword="", 
        spatial_grid_1=0, 
        spatial_grid_2=0, 
        spatial_grid_3=0, 
        out_alias=""
        )

    # Process: Append Polygons (Append) (management)
    polygons_append = arcpy.management.Append(
        inputs=[
                nfpors_fuels_treatments_enriched, 
                Tahoe_Forest_Fuels_Tx_enriched, 
                usfs_edw_facts_common_attributes_enriched, 
                nps_flat_fuels_enriched, 
                CNRA_Enriched_poly, 
                BLM_enriched], 
        target=polygons, 
        schema_type="TEST", 
        field_mapping="", 
        subtype="", 
        expression=""
        )

    # Process: 2b Assign Domains (3) (2b Assign Domains) (PC414CWIMillionAcres)
    polygons_append_ad = AssignDomains(in_table=polygons_append)

    # Assign Metadata to the featureclass
    polygons_append_md = poly_metadata(polygons_append_ad) 


ExecuteError: ERROR 000466: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\d_Enriched\nfpors_fuels_treatments_enriched_20220906 does not match the schema of target c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\e_Appended\Treat_n_harvests_polygons_20240124
Failed to execute (Append).
